# Stromverbrauch

Wir binden zunächst allgemeine Einstellungen ein, die für alle Notebooks gültig sind.

In [ ]:
%run general_properties.ipynb

## Beispielausgabe

<img src="Stromverbrauch.png" alt="Drawing" style="width: 90%; max-width: 1000px; margin-left: auto; margin-right: auto;"/>

## Daten einlesen

In [ ]:
# Laden der minütlichen Information
data = pd.read_csv('example_data/example.csv', delimiter=",",index_col=0, parse_dates=[0], na_values=['No CT'])
#data

## Gesamtenergieverbrauch Tagesverlauf (optional summiert über mehrere Tage)
Wir wollen wissen, zu welchen Uhrzeiten wir typischerweise viel Strom verbrauchen. 
Diese Auswertung kann über einen Tag geschehen, oder summiert / gemittelt über mehrere Tage.

## Daten eingrenzen

Wir beschränken die verfügbaren Daten zunächst auf einen Zeitraum:

In [ ]:
plotdata = data["2022-07-09":"2022-07-11"]

Da wir die Daten später gruppieren möchten, wandeln wir die Index-Spalte (momentan die Zeit) in eine normale Spalte um.

In [ ]:
plotdata = plotdata.reset_index()

## Daten gruppieren
Gruppiert wird nun auf Tag und Stunde. Als Funktion wenden wir den Mittelwert auf die gruppierten Daten an und verwenden apply, um etwas flexibler zu sein.

In [ ]:
gdata = plotdata.groupby([plotdata.iloc[:,0].dt.day, plotdata.iloc[:,0].dt.hour]).apply(lambda s: pd.Series({ 
    "HV1": s["Hauptverteiler_1 (kWatts)"].mean(), 
    "HV2": s["Hauptverteiler_2 (kWatts)"].mean(), 
    "HV3": s["Hauptverteiler_3 (kWatts)"].mean(), 
    "HV": (s["Hauptverteiler_1 (kWatts)"]+s["Hauptverteiler_2 (kWatts)"]+s["Hauptverteiler_3 (kWatts)"]).mean(), 
}))

Nachdem wir den Tagesverlauf noch einzeln für jeden Tag im Abfragezeitraum haben, können wir eine nächste Gruppierung anwenden, um mehrere Tage gleichzeitg auszuwerten. Wir gruppieren also nur noch auf Stunden. Dazu müssen wir zunächst den multiindex aus der vorherigen, zweifachen Gruppierung auf die Zeitspalte auflösen. Das tun wir, in dem die Indexspalten umbenannt werden. (Zuvor hatten beide den gleichen Namen, was zu fehlern beim Auflösen des Index führt.)

In [ ]:
# set index names for multiindex
gdata.index = gdata.index.set_names(['Day', 'Hour'])
# convert index to normal columns
gdata2 = gdata.reset_index()
# summed values
gdata2sum = gdata2.groupby([gdata2.iloc[:,1]])[['HV1','HV2','HV3']].sum()
# mean values
gdata2mean = gdata2.groupby([gdata2.iloc[:,1]])[['HV1','HV2','HV3']].mean()

### Daten darstellen

In [ ]:
h = plt.figure(figsize=(10, 5))
ax = h.gca()
gdata2mean.plot.bar(stacked=True, rot=90, ax=ax)
ax.set_title("Akkumulierte Stundenwerte",font='DejaVu Sans', size=12)
ax.set_xlabel("Stunde")
ax.set_ylabel("Stromverbrauch in kW")
#h.savefig("Stromverbrauch.pdf")